# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [1]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
OLD_SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-perps'
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [3]:
# data contract
PerpsV2DataAbi = """
[
    {
      "inputs": [
        {
          "internalType": "contract IAddressResolver",
          "name": "_resolverProxy",
          "type": "address"
        }
      ],
      "payable": false,
      "stateMutability": "nonpayable",
      "type": "constructor"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "allMarketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "allProxiedMarketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "globals",
      "outputs": [
        {
          "components": [
            {
              "internalType": "uint256",
              "name": "minInitialMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationFeeRatio",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationBufferRatio",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "minKeeperFee",
              "type": "uint256"
            }
          ],
          "internalType": "struct PerpsV2MarketData.FuturesGlobals",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "contract IPerpsV2MarketViews",
          "name": "market",
          "type": "address"
        }
      ],
      "name": "marketDetails",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "baseAsset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "marketKey",
              "type": "bytes32"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxLeverage",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "maxMarketValue",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketLimits",
              "name": "limits",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxFundingVelocity",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "skewScale",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FundingParameters",
              "name": "fundingParameters",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "marketSize",
                  "type": "uint256"
                },
                {
                  "components": [
                    {
                      "internalType": "uint256",
                      "name": "long",
                      "type": "uint256"
                    },
                    {
                      "internalType": "uint256",
                      "name": "short",
                      "type": "uint256"
                    }
                  ],
                  "internalType": "struct PerpsV2MarketData.Sides",
                  "name": "sides",
                  "type": "tuple"
                },
                {
                  "internalType": "uint256",
                  "name": "marketDebt",
                  "type": "uint256"
                },
                {
                  "internalType": "int256",
                  "name": "marketSkew",
                  "type": "int256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketSizeDetails",
              "name": "marketSizeDetails",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "price",
                  "type": "uint256"
                },
                {
                  "internalType": "bool",
                  "name": "invalid",
                  "type": "bool"
                }
              ],
              "internalType": "struct PerpsV2MarketData.PriceDetails",
              "name": "priceDetails",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        }
      ],
      "name": "marketDetailsForKey",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "baseAsset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "marketKey",
              "type": "bytes32"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxLeverage",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "maxMarketValue",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketLimits",
              "name": "limits",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxFundingVelocity",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "skewScale",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FundingParameters",
              "name": "fundingParameters",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "marketSize",
                  "type": "uint256"
                },
                {
                  "components": [
                    {
                      "internalType": "uint256",
                      "name": "long",
                      "type": "uint256"
                    },
                    {
                      "internalType": "uint256",
                      "name": "short",
                      "type": "uint256"
                    }
                  ],
                  "internalType": "struct PerpsV2MarketData.Sides",
                  "name": "sides",
                  "type": "tuple"
                },
                {
                  "internalType": "uint256",
                  "name": "marketDebt",
                  "type": "uint256"
                },
                {
                  "internalType": "int256",
                  "name": "marketSkew",
                  "type": "int256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketSizeDetails",
              "name": "marketSizeDetails",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "price",
                  "type": "uint256"
                },
                {
                  "internalType": "bool",
                  "name": "invalid",
                  "type": "bool"
                }
              ],
              "internalType": "struct PerpsV2MarketData.PriceDetails",
              "name": "priceDetails",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "address[]",
          "name": "markets",
          "type": "address[]"
        }
      ],
      "name": "marketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32[]",
          "name": "marketKeys",
          "type": "bytes32[]"
        }
      ],
      "name": "marketSummariesForKeys",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        }
      ],
      "name": "parameters",
      "outputs": [
        {
          "components": [
            {
              "internalType": "uint256",
              "name": "takerFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "overrideCommitFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "takerFeeDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFeeDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "takerFeeOffchainDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFeeOffchainDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxMarketValue",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxFundingVelocity",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "skewScale",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "nextPriceConfirmWindow",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "delayedOrderConfirmWindow",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "minDelayTimeDelta",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxDelayTimeDelta",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "offchainDelayedOrderMinAge",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "offchainDelayedOrderMaxAge",
              "type": "uint256"
            },
            {
              "internalType": "bytes32",
              "name": "offchainMarketKey",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "offchainPriceDivergence",
              "type": "uint256"
            }
          ],
          "internalType": "struct IPerpsV2MarketSettings.Parameters",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "contract IPerpsV2MarketViews",
          "name": "market",
          "type": "address"
        },
        {
          "internalType": "address",
          "name": "account",
          "type": "address"
        }
      ],
      "name": "positionDetails",
      "outputs": [
        {
          "components": [
            {
              "components": [
                {
                  "internalType": "uint64",
                  "name": "id",
                  "type": "uint64"
                },
                {
                  "internalType": "uint64",
                  "name": "lastFundingIndex",
                  "type": "uint64"
                },
                {
                  "internalType": "uint128",
                  "name": "margin",
                  "type": "uint128"
                },
                {
                  "internalType": "uint128",
                  "name": "lastPrice",
                  "type": "uint128"
                },
                {
                  "internalType": "int128",
                  "name": "size",
                  "type": "int128"
                }
              ],
              "internalType": "struct IPerpsV2MarketBaseTypes.Position",
              "name": "position",
              "type": "tuple"
            },
            {
              "internalType": "int256",
              "name": "notionalValue",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "profitLoss",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "accruedFunding",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "remainingMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "accessibleMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationPrice",
              "type": "uint256"
            },
            {
              "internalType": "bool",
              "name": "canLiquidatePosition",
              "type": "bool"
            }
          ],
          "internalType": "struct PerpsV2MarketData.PositionData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        },
        {
          "internalType": "address",
          "name": "account",
          "type": "address"
        }
      ],
      "name": "positionDetailsForMarketKey",
      "outputs": [
        {
          "components": [
            {
              "components": [
                {
                  "internalType": "uint64",
                  "name": "id",
                  "type": "uint64"
                },
                {
                  "internalType": "uint64",
                  "name": "lastFundingIndex",
                  "type": "uint64"
                },
                {
                  "internalType": "uint128",
                  "name": "margin",
                  "type": "uint128"
                },
                {
                  "internalType": "uint128",
                  "name": "lastPrice",
                  "type": "uint128"
                },
                {
                  "internalType": "int128",
                  "name": "size",
                  "type": "int128"
                }
              ],
              "internalType": "struct IPerpsV2MarketBaseTypes.Position",
              "name": "position",
              "type": "tuple"
            },
            {
              "internalType": "int256",
              "name": "notionalValue",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "profitLoss",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "accruedFunding",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "remainingMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "accessibleMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationPrice",
              "type": "uint256"
            },
            {
              "internalType": "bool",
              "name": "canLiquidatePosition",
              "type": "bool"
            }
          ],
          "internalType": "struct PerpsV2MarketData.PositionData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "resolverProxy",
      "outputs": [
        {
          "internalType": "contract IAddressResolver",
          "name": "",
          "type": "address"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    }
  ]
"""

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


In [4]:
# functions
def convertDecimals(x): return Decimal(x) / Decimal(10**18)


def convertBytes(x): return bytearray.fromhex(
    x[2:]).decode().replace('\x00', '')


def clean_df(df, decimal_cols=[], bytes_cols=[]):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    for col in bytes_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertBytes)
        else:
            print(f"{col} not in DataFrame")
    return df


async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df


async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True

    df = pd.DataFrame(all_results)
    return df


In [5]:
transfer_query = gql("""
query transfers(
    $last_id: ID!
  ) {
		futuresMarginTransfers(
      where: {
        id_gt: $last_id
      }
			first: 1000
		) {
			id
			timestamp
			account
			market
			size
			asset
			txHash
		}
}
""")
                     
trade_query = gql("""
query trades(
    $last_id: ID!
  ) {
    futuresTrades(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      timestamp
      asset
      marketKey
      orderType
      price
      margin
      positionId
      feesPaid
      pnl
      size
      positionSize
      positionClosed
      trackingCode
    }  
}
""")


## Get transfers and trades

In [6]:
trade_params = {
    'last_id': ''
}

trade_decimal_cols = [
    'price',
    'margin',
    'feesPaid',
    'pnl',
    'size',
    'positionSize',
]

trade_bytes_cols = ['asset']

df_trade = await run_recursive_query(trade_query, trade_params, 'futuresTrades')
df_trade = df_trade.replace({None: np.nan})
df_trade = clean_df(df_trade, decimal_cols=trade_decimal_cols, bytes_cols=trade_bytes_cols)

In [7]:
transfer_params = {
    'last_id': ''
}

transfer_cols = [
    'size',
]

transfer_bytes_cols = [
    'asset'
]

df_transfer = await run_recursive_query(transfer_query, transfer_params, 'futuresMarginTransfers')
df_transfer = df_transfer.replace({None: np.nan})
df_transfer = clean_df(df_transfer, decimal_cols=transfer_cols, bytes_cols=transfer_bytes_cols)


In [8]:
## summarize transfers
df_transfer_summ = df_transfer.groupby('asset')['size'].sum().reset_index()
df_transfer_summ.columns = ['asset', 'netTransfers']

## summarize trades
df_trade_summ = df_trade.groupby('asset')['feesPaid'].sum().reset_index()
df_trade_summ.columns = ['asset', 'feesPaid']


In [9]:
df_trade_summ

,asset,feesPaid
0,AAVE,5185.335300981038694606
1,APE,32370.743034667448817069
2,ARB,56722.935972129101790392
3,ATOM,31521.238123718427363205
4,AUD,117.687263282130204308
5,AVAX,48817.596449858583354467
6,AXS,2906.792852056191430969
7,BNB,76390.624826628093559220
8,DOGE,15671.448093534120949685
9,DYDX,41764.034466916573709712


## Get the market data

In [10]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)
marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call()



In [11]:
markets = [{
    'asset': market[1].decode().replace('\x00', ''),
    'marketDebt': w3.fromWei(market[7], unit='ether')
} for market in marketSummaries]

df_markets = pd.DataFrame(markets)
df_markets

,asset,marketDebt
0,sETH,6572839.191384058457137087
1,sBTC,6826410.370596348249874329
2,LINK,247758.491690433157728465
3,SOL,423557.972579813527821882
4,AVAX,298305.63810470615359472
5,AAVE,3398.895454652178904028
6,UNI,94163.45848645428623296
7,MATIC,370982.795568991890374637
8,APE,84442.962650731599269394
9,DYDX,34359.651931747648251722


In [12]:
df_pnl = df_markets.merge(df_transfer_summ, on='asset').merge(df_trade_summ, on='asset')
df_pnl['netPnl'] = df_pnl['marketDebt'] - df_pnl['netTransfers'] - df_pnl['feesPaid']
df_pnl

,asset,marketDebt,netTransfers,feesPaid,netPnl
0,sETH,6572839.191384058457137087,7659102.356595678200519030,1159497.972004160702960434,-2245761.137215780446342377
1,sBTC,6826410.370596348249874329,8214234.746435687193598221,1191558.143638584728791897,-2579382.519477923672515789
2,LINK,247758.491690433157728465,364396.528943698637083790,119077.342891368386868816,-235715.380144633866224141
3,SOL,423557.972579813527821882,557580.459278165025606784,130743.351428245224276188,-264765.838126596722061090
4,AVAX,298305.63810470615359472,219511.502625528761844985,48817.596449858583354467,29976.539029318808395268
5,AAVE,3398.895454652178904028,1904.010032449485404462,5185.335300981038694606,-3690.449878778345195040
6,UNI,94163.45848645428623296,108874.028780298292392652,11870.841556325537115514,-26581.411850169543275206
7,MATIC,370982.795568991890374637,513418.048015365130663223,108237.037814745497935859,-250672.290261118738224445
8,APE,84442.962650731599269394,56592.134348969888103569,32370.743034667448817069,-4519.914732905737651244
9,DYDX,34359.651931747648251722,71192.633786560074534017,41764.034466916573709712,-78597.016321728999992007


In [13]:
df_pnl['netPnl'].sum()

Decimal('-6551993.530034809342679764')

In [14]:
df_pnl['netPnlClean'] = df_pnl['netPnl'].apply(lambda x: round(x, 2))
df_pnl[['asset', 'netPnlClean']].sort_values('netPnlClean', ascending=True)

,asset,netPnlClean
1,sBTC,-2579382.52
0,sETH,-2245761.14
11,OP,-362713.24
3,SOL,-264765.84
7,MATIC,-250672.29
2,LINK,-235715.38
19,FTM,-164508.28
23,ARB,-138890.86
10,BNB,-121662.48
9,DYDX,-78597.02
